In [ ]:
%load_ext sparkmagic.magics
from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy()
dsxhi_util.list_livy_endpoints()

success configuring sparkmagic livy.
['https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1', 'https://qlawsdl001038a.ad.datalake.foc.zone:8443/gateway/dsx/livy/v1']


In [ ]:
%%spark config
{"executorCores": 4, "numExecutors": 5, "executorMemory": "10g",
 "driverMemory": "8g", "proxyUser": "jlu-", "driverCores": 1,
 "conf": {"spark.yarn.appMasterEnv.THEANO_FLAGS": "base_compiledir=${PWD}/.theano"}}


In [ ]:
%spark add -s refimodel -k -l python -u https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
36693,application_1617092791322_350537,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [ ]:
%%spark

print(spark.version)

2.3.0.2.6.5.0-292

In [ ]:
%%spark

import pyspark
import os, sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.functions import length, col, when, count, countDistinct, mean, lit, sum, udf, desc, min, max
from pyspark.sql import DataFrameStatFunctions as statFunc
from pyspark.sql.types import *
from pyspark.sql.functions import rand
from pyspark.sql.functions import monotonically_increasing_id

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier

from pyspark.ml.feature import VectorIndexer, VectorAssembler, VectorSlicer, StringIndexer, QuantileDiscretizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

import os
import datetime
import time
import pandas as pd
import numpy as np

spark = SparkSession.builder.getOrCreate()

In [ ]:
%%spark

model_save_path = '/uat/DataScience/Maestro/Mover Refi/Model/'
training_data_path = '/dev/projects/retention_models/refi_payoff/training/data/train'
test_data_path = '/dev/projects/retention_models/refi_payoff/training/data/test'
result_path = '/dev/projects/retention_models/refi_payoff/training/results/'

monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'

# 1. Load Data

### Preprocessed Data

In [ ]:
%%spark

# df201612 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201612.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
# df201703 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201703.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
# df201706 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201706.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
# df201709 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201709.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
# df201712 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201712.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
# df201803 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201803.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
# df201806 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201806.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
# df201809 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201809.parquet').drop('gcid', 'p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201812 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201812.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201903 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201903.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201906 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201906.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201909 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201909.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df201912 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201912.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202003 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202003.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202006 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202006.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202009 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202009.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202012 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202012.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')


df_preprocessed = df201812.union(df201903).union(df201906).union(df201909).union(df201912)\
                          .union(df202003).union(df202006).union(df202009).union(df202012)\
                          .drop('borrowerage_bucket','p_married_exp','p_31_50_exp')

In [ ]:
%%spark

target = 'refipayoff'

### Actual Payoff Data

In [ ]:
%%spark

actual = spark.read.parquet('/uat/DataScience/Maestro/Mover Refi/Actual_payoff_20210405.parquet')\
                .select('servicecalendardate', 'ln_no', 'refipayoff')

# actual.orderBy('servicecalendardate').show(1)

## Additional Features

In [ ]:
%%spark
addcol = 'select * from domain_internal_leadgen.lead_created_killed_state'
addcol = spark.sql(addcol).select('loannumber','deniedorwithdrawn','killedreason')
addcol = addcol.withColumnRenamed('loannumber','ln_no')
# addcol.show(5)

pty = spark.table('servicing_raw_access.servicingmsp_ptyfile_master_vw') \
            .where(F.col('delta_file_byte') != 'D') \
            .withColumnRenamed('ln_no', 'loannumber') \
            .withColumnRenamed('pr_alpha_state_cd', 'pty_state') \
            .withColumnRenamed('loannumber','ln_no')\
            .select('ln_no', 'pty_state') \
            .dropDuplicates()
# pty.show(5)

rms = spark.table('domain_internal_leadgen.client_web_behavior_raw_ql_rms')\
            .where(F.col('omnidatetime')>'2018-01-01 00:00:00')\
            .withColumn('cancelflg', F.when(~F.col('omnidatetime').isNull(), 'yes').otherwise('no'))\
            .withColumnRenamed('loan_number','ln_no')\
            .select('ln_no','cancelflg')\
            .dropDuplicates()
# rms.show(5)

bbdat = 'select * from domain_internal_leadgen.bigbang_mover_prep'
bbdat = spark.sql(bbdat).select('loan_number','is_viable')
bbdat = bbdat.withColumnRenamed('loan_number','ln_no')
# bbdat.show(5)

### Join actual data with preprocessed data

In [ ]:
%%spark

df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner')\
                        .join(addcol, on=['ln_no'], how='left')\
                        .join(rms, on=['ln_no'], how='left')\
                        .join(bbdat, on=['ln_no'], how='left').dropDuplicates()
#  .join(pty, on=['ln_no'], how='left')\
# df_all.show(5)

# df_all.where((F.col('servicecalendardate')>'2019-07-11') & (F.col('is_viable').isNull())).select('ln_no').show(10)
# df_all.show(5)
# df_all.groupby('servicecalendardate').agg(count('ln_no'), sum(target)).orderBy(col('servicecalendardate')).show()

In [ ]:
%%spark

df_all.groupby('refipayoff').count().show()
df_all.groupby('deniedorwithdrawn').count().show()
df_all.groupby('killedreason').count().show()
df_all.groupby('cancelflg').count().show()
df_all.groupby('pty_state').count().show(10)

+----------+--------+
|refipayoff|   count|
+----------+--------+
|       0.0|15988406|
|       1.0|  931898|
+----------+--------+

+-----------------+--------+
|deniedorwithdrawn|   count|
+-----------------+--------+
|             null|16624773|
|           Denied|   29392|
| Withdrawn_others|   98012|
|        Withdrawn|  168127|
+-----------------+--------+

+--------------------+--------+
|        killedreason|   count|
+--------------------+--------+
|Invalid Phone Num...|     279|
|      Excessive debt|     308|
|  Future Opportunity|    8797|
|  Received Lead Late|       5|
|       Withdraw lead|    3195|
|       Property type|      44|
|Self employed inc...|      46|
|Invalid Email Add...|      86|
|Client Dialed Wro...|     483|
|Competitor - Rela...|     129|
|  No Mortgage Needed|     184|
|                null|16624791|
|Future Purchase O...|    3580|
|Client Not Intere...|  118913|
|  Insufficient funds|       7|
|Duplicate Lead Al...|     821|
|  Home Has Been Sold|    

In [ ]:
%%spark

st = ['SC','AZ','LA','MN','NJ','OR','VA','RI','KY','WY','NH','MI','NV','WI','ID','CA','CT','NE','MT','NC','VT','MD','DE','MO','IL',
      'ME','WA','ND','MS','AL','IN','OH','TN','NM','IA','PA','SD','NY','TX','WV','GA','MA','KS','CO','FL','AK','AR','OK','UT','HI']

def preprocess2(df):
#     state_rt = when(col('pty_state').isin(st), col('pty_state')).otherwise('missing')
    cancel_rt = when(col('cancelflg').isNull(), 'no').otherwise(F.col('cancelflg'))
    reason_rt = when(col('killedreason').isNull(), 'missing').otherwise(col('killedreason'))
    dw_rt = when(col('deniedorwithdrawn').isNull(), 'missing').otherwise(col('deniedorwithdrawn'))
    house_rt = when(col('home_value_exp') < 100000, '<100k')\
               .when((col('home_value_exp') >= 100000) & (col('home_value_exp') < 300000), '100k-300k')\
               .when((col('home_value_exp') >= 300000) & (col('home_value_exp') < 600000), '300k-600k')\
               .when((col('home_value_exp') >= 600000) & (col('home_value_exp') < 900000), '600k-900k')\
               .when((col('home_value_exp') >= 900000) & (col('home_value_exp') < 1200000), '900k-1.2m')\
               .when(col('home_value_exp') >= 1200000, '>=1.2m')
    loan_rt = when(col('og_mtg_am_exp') < 100000, '<100k')\
               .when((col('og_mtg_am_exp') >= 100000) & (col('og_mtg_am_exp') < 300000), '100k-300k')\
               .when((col('og_mtg_am_exp') >= 300000) & (col('og_mtg_am_exp') < 600000), '300k-600k')\
               .when((col('og_mtg_am_exp') >= 600000) & (col('og_mtg_am_exp') < 900000), '600k-900k')\
               .when((col('og_mtg_am_exp') >= 900000) & (col('og_mtg_am_exp') < 1200000), '900k-1.2m')\
               .when(col('og_mtg_am_exp') >= 1200000, '>=1.2m')
    viable_rt = when(col('is_viable') == 1, 'yes')\
               .when(col('is_viable') == 0, 'no')\
               .when(col('is_viable').isNull(), 'missing')

#                 .withColumn('pty_state', state_rt)\
    df_final = df.withColumn('cancelflg', cancel_rt)\
                 .withColumn('killedreason', reason_rt)\
                 .withColumn('deniedorwithdrawn', dw_rt)\
                 .withColumn('home_value_exp', house_rt)\
                 .withColumn('og_mtg_am_exp', loan_rt)\
                 .withColumn('is_viable', viable_rt)\
                 .withColumn('ct_1_exp', F.col('ct_1_exp').cast(DoubleType()))\
                 .withColumn('ct_2_exp', F.col('ct_2_exp').cast(DoubleType()))\
                 .withColumn('ct_3_exp', F.col('ct_3_exp').cast(DoubleType()))\
                 .withColumn('ct_age_exp', F.col('ct_age_exp')+3)\
                 .withColumn('ln_ann_int_rt', F.col('ln_ann_int_rt').cast(DoubleType()))\
                 .drop('og_note_dt', 'mosaic_group_pur_exp', 'ratespread_min_pur_exp',
                       'ln_purpose_type', 'investornameshort')

    return df_final

df_all = preprocess2(df_all).drop('killedreason','deniedorwithdrawn','ct_3_exp','LiveYears_grp_exp')

df_all.show(5)

+----------+-------------------+----------+--------+--------+--------------+------------+----------------------+---------------------+------------------+-------------+-----------------------+--------------------+------------+-------------+------------------+-------------+-------------------+------------------+-------------------+-----------------------+--------------------+---------+----------+---------+---------+
|     ln_no|servicecalendardate|ct_age_exp|ct_1_exp|ct_2_exp|home_value_exp|p_edu_hs_exp|personnum_per_room_exp|mosaic_group_refi_exp|ratespread_min_exp|ln_ann_int_rt|loantypedescription_exp|loanamortizationtype|ageinmon_exp|og_mtg_am_exp|   currentcltv_exp|orig_fico_exp|LiveYears_short_exp|LiveYears_long_exp|ln_purpose_type_exp|og_occupy_stat_type_exp|issingleborrower_exp|ln_tr_exp|refipayoff|cancelflg|is_viable|
+----------+-------------------+----------+--------+--------+--------------+------------+----------------------+---------------------+------------------+-----------

In [ ]:
%%spark
# df_all.repartition(1).write.save(path='hdfs://datalakeprod/uat/DataScience/Maestro/Mover Refi/df_reducedfeatures.csv', format='csv', mode='overwrite', header='true')

In [ ]:
%%spark
# df_all = spark.read.csv(path='hdfs://datalakeprod/uat/DataScience/Maestro/Mover Refi/df_reducedfeatures.csv',header=True)
# target = 'refipayoff'
# num_cols = ['refipayoff','ct_age_exp', 'ct_1_exp', 'ct_2_exp', 'p_edu_hs_exp', 'personnum_per_room_exp', 'ratespread_min_exp', 'ln_ann_int_rt', 'ageinmon_exp', 'currentcltv_exp', 'orig_fico_exp', 'LiveYears_short_exp', 'LiveYears_long_exp', 'issingleborrower_exp', 'ln_tr_exp']
# for i in num_cols:
#     df_all = df_all.withColumn(i, F.col(i).cast(DoubleType()))
# df_all = df_all.withColumn('refipayoff', F.col('refipayoff').cast(IntegerType()))

In [ ]:
%%spark
exclude_cols = (target)
num_cols = [i[0] for i in df_all.dtypes if ((i[1].startswith(('int', 'double'))) & (~i[0].endswith(exclude_cols)))]
cat_cols = [i[0] for i in df_all.dtypes if ((i[1]=='string') & (~i[0].endswith(exclude_cols)))]

In [ ]:
%%spark
num_cols

['ct_age_exp', 'ct_1_exp', 'ct_2_exp', 'p_edu_hs_exp', 'personnum_per_room_exp', 'ratespread_min_exp', 'ln_ann_int_rt', 'ageinmon_exp', 'currentcltv_exp', 'orig_fico_exp', 'LiveYears_short_exp', 'LiveYears_long_exp', 'issingleborrower_exp', 'ln_tr_exp']

In [ ]:
%%spark
cat_cols

['ln_no', 'servicecalendardate', 'home_value_exp', 'mosaic_group_refi_exp', 'loantypedescription_exp', 'loanamortizationtype', 'og_mtg_am_exp', 'ln_purpose_type_exp', 'og_occupy_stat_type_exp', 'cancelflg', 'is_viable']

# 2. Build Models

### Sample Data

In [ ]:
%%spark

# create fundtion for undersampling
def data_split(df, train_months, target, rate):

    ## Split Train/Validation From Test ##
    train_validate = df.where(col('servicecalendardate').isin(train_months))

    ## Split Out Payoffs into Train/Validate ##
    train_refi_all = train_validate.filter(train_validate[target] == '1.')
    train_refi, validate_refi = train_refi_all.randomSplit([0.7, 0.3], seed=123)

    ## Split Out Non-Payoffs to Train/Validate
    train_nonrefi_all = train_validate.filter(train_validate[target] == '0.')
    train_nonrefi, validate_nonrefi = train_nonrefi_all.randomSplit([0.7, 0.3], seed=123)

    ## Undersample Non-Payoffs ##
    train_non, notused1 = train_nonrefi.randomSplit([rate, 1-rate], seed=123)
    val_non, notused2 = validate_nonrefi.randomSplit([rate, 1-rate], seed=123)

    ## Create Final Training/Validate Sets
    train = train_refi.unionAll(train_non)
    validate = validate_refi.unionAll(val_non)
    train.cache()

    return train, validate

In [ ]:
%%spark

train_months = ['2018-12-31', '2019-03-31', '2019-06-30', '2019-09-30', '2019-12-31', '2020-03-31', '2020-06-30']

df_train, df_validate = data_split(df_all, train_months, target, 0.05)

In [ ]:
%%spark

df_train.groupby(['servicecalendardate', target]).count().orderBy('servicecalendardate', target).show()

+-------------------+----------+------+
|servicecalendardate|refipayoff| count|
+-------------------+----------+------+
|         2018-12-31|       0.0| 58997|
|         2018-12-31|       1.0| 25932|
|         2019-03-31|       0.0| 60053|
|         2019-03-31|       1.0| 33688|
|         2019-06-30|       0.0| 61319|
|         2019-06-30|       1.0| 53671|
|         2019-09-30|       0.0| 58740|
|         2019-09-30|       1.0| 65471|
|         2019-12-31|       0.0| 59785|
|         2019-12-31|       1.0| 71178|
|         2020-03-31|       0.0| 60691|
|         2020-03-31|       1.0| 95069|
|         2020-06-30|       0.0| 62247|
|         2020-06-30|       1.0|107490|
+-------------------+----------+------+

### Boruta

In [ ]:
%%spark

def _create_shadow(x):
    """
    Take all X variables, creating copies and randomly shuffling them
    :param x: the dataframe to create shadow features on
    :return: dataframe 2x width and the names of the shadows for removing later
    """
    shadow_dat = {}
    x_shadow = x
    w = Window.orderBy('rand')
    x = x.withColumn('rand',rand()).withColumn('id', F.row_number().over(w)).drop('rand')

    for c in x_shadow.columns:
        shadow_dat[c+'_shadow'] = x_shadow.select(c).withColumn('rand',rand())\
                                                    .withColumn('id', F.row_number().over(w))\
                                                    .withColumnRenamed(c, c+'_shadow').drop('rand') # shuffle the values of each feature to all the features
        x = x.join(shadow_dat[c+'_shadow'], on=['id'], how='left')
    return x

shad_train = df_train.drop('killedreason','ct_3_exp','ln_no','servicecalendardate',\
                           'LiveYears_grp_exp','deniedorwithdrawn')
shad_dat = _create_shadow(shad_train).drop('id','refipayoff_shadow')
shad_dat.show()

+----------+--------+--------+--------------+------------+----------------------+---------------------+-------------------+-------------+-----------------------+--------------------+------------+-------------+------------------+-------------+-------------------+------------------+-------------------+-----------------------+--------------------+---------+----------+---------+---------+---------+-----------------+---------------+---------------+---------------------+-------------------+-----------------------------+----------------------------+-------------------------+--------------------+------------------------------+---------------------------+-------------------+--------------------+----------------------+--------------------+--------------------------+-------------------------+--------------------------+------------------------------+---------------------------+----------------+----------------+----------------+----------------+
|ct_age_exp|ct_1_exp|ct_2_exp|home_value_exp|p_edu_hs

In [ ]:
%%spark

def create_pipeline(target, *arg):

    exclude_cols = ('servicecalendardate','ln_no',target)
    cat_cols = [i[0] for i in shad_dat.dtypes if ((i[1]=='string') & (~i[0].endswith(exclude_cols)))]
    num_cols = [i[0] for i in shad_dat.dtypes if ((i[1].startswith(('int', 'double'))) & (~i[0].endswith(exclude_cols)))]

    stages = []

    for col in cat_cols:

        #Category indexing with StringIndexer
        indexer = StringIndexer(inputCol = col, outputCol = col+'_idx').setHandleInvalid('keep')
        stages += [indexer]

    #assemblerInputs = [c+'_vec' for c in cat_cols] + num_cols
    assemblerInputs = [c+'_idx' for c in cat_cols] + num_cols
    assembler = VectorAssembler(inputCols = assemblerInputs, outputCol = 'vectFeatures')

    stages += [assembler]

    lr = LogisticRegression(maxIter=100, regParam=0.1, elasticNetParam=0.0, fitIntercept = True,
                            featuresCol='vectFeatures', labelCol=target)
    rf = RandomForestClassifier(maxBins=52, numTrees=250, maxDepth = 5, featuresCol='vectFeatures', labelCol=target)
    gbt = GBTClassifier(maxBins=52, maxIter=100, featuresCol='vectFeatures', labelCol=target)

    pipeline_lr = Pipeline(stages = stages + [lr])
    pipeline_rf = Pipeline(stages = stages + [rf])
    pipeline_gbt = Pipeline(stages = stages + [gbt])

    if "lr" in arg and "rf" in arg and "gbt" in arg:
        return lr, rf, gbt, pipeline_lr, pipeline_rf, pipeline_gbt
    elif "lr" in arg and "rf" in arg:
        return lr, rf, pipeline_lr, pipeline_rf
    elif "lr" in arg and "gbt" in arg:
        return lr, gbt, pipeline_lr, pipeline_gbt
    elif "rf" in arg and "gbt" in arg:
        return rf, gbt, pipeline_rf, pipeline_gbt
    elif "rf" in arg:
        return rf, pipeline_rf
    elif "lr" in arg:
        return lr, pipeline_lr
    elif "gbt" in arg:
        return gbt, pipeline_gbt
    else:
        return gbt, pipeline_gbt

def fit_pipeline(pipeline, training_dataset):
    model_pipeline = pipeline.fit(training_dataset)
    return model_pipeline

def persist_modelpersist_(pipeline_model, model_name, model_save_path=model_save_path):
    pipeline_model.write().overwrite().save(model_save_path + model_name)

In [ ]:
%%spark
lr, rf, gbt, pipeline_lr, pipeline_rf, pipeline_gbt = create_pipeline(target, "lr", "rf", "gbt")

# Gradient Boosting
start = time.time()
model_pipeline_gbt = fit_pipeline(pipeline_gbt, shad_dat)
end = time.time()

print('Gradient Boosting Training Time:', end - start)

Gradient Boosting Training Time: 1747.3582675457

In [ ]:
%%spark

def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

def predict(model_pipeline, data):

    _scoreUdf = udf(lambda v: float(v[1]), DoubleType())

    prediction = model_pipeline.transform(data)
    pred_df = prediction.withColumn('pred', _scoreUdf(prediction['probability']))
    return pred_df

def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [ ]:
%%spark

pred_df_gbt = predict(model_pipeline_gbt, shad_dat)
score = ExtractFeatureImp(model_pipeline_gbt.stages[-1].featureImportances, pred_df_gbt, "vectFeatures").score.tolist()
name = ExtractFeatureImp(model_pipeline_gbt.stages[-1].featureImportances, pred_df_gbt, "vectFeatures").name.tolist()

a = {}
for i in range(len(score)):
    a[name[i]] = score[i]

a = sorted(a.items())

# Excluded Columns
for i in range(len(a)):
    if i%2 == 0:
        if a[i][1] <= a[i+1][1]:
            print(a[i],a[i+1])

Excluded Columns:

### Creat Pipeline

In [ ]:
%%spark

def create_pipeline(target, *arg):

    exclude_cols = ('servicecalendardate','ln_no',target)
    cat_cols = [i[0] for i in df_train.dtypes if ((i[1]=='string') & (~i[0].endswith(exclude_cols)))]
    num_cols = [i[0] for i in df_train.dtypes if ((i[1].startswith(('int', 'double'))) & (~i[0].endswith(exclude_cols)))]

    stages = []

    for col in cat_cols:

        #Category indexing with StringIndexer
        indexer = StringIndexer(inputCol = col, outputCol = col+'_idx').setHandleInvalid('keep')
        stages += [indexer]

    #assemblerInputs = [c+'_vec' for c in cat_cols] + num_cols
    assemblerInputs = [c+'_idx' for c in cat_cols] + num_cols
    assembler = VectorAssembler(inputCols = assemblerInputs, outputCol = 'vectFeatures')

    stages += [assembler]

    lr = LogisticRegression(maxIter=100, regParam=0.1, elasticNetParam=0.0, fitIntercept = True,
                            featuresCol='vectFeatures', labelCol=target)
    rf = RandomForestClassifier(maxBins=52, numTrees=250, maxDepth = 5, featuresCol='vectFeatures', labelCol=target)
    gbt = GBTClassifier(maxIter=150, maxDepth=5, featuresCol='vectFeatures', labelCol=target) #maxIter=100, maxDepth=10, maxBins=50

    pipeline_lr = Pipeline(stages = stages + [lr])
    pipeline_rf = Pipeline(stages = stages + [rf])
    pipeline_gbt = Pipeline(stages = stages + [gbt])

    if "lr" in arg and "rf" in arg and "gbt" in arg:
        return lr, rf, gbt, pipeline_lr, pipeline_rf, pipeline_gbt
    elif "lr" in arg and "rf" in arg:
        return lr, rf, pipeline_lr, pipeline_rf
    elif "lr" in arg and "gbt" in arg:
        return lr, gbt, pipeline_lr, pipeline_gbt
    elif "rf" in arg and "gbt" in arg:
        return rf, gbt, pipeline_rf, pipeline_gbt
    elif "rf" in arg:
        return rf, pipeline_rf
    elif "lr" in arg:
        return lr, pipeline_lr
    elif "gbt" in arg:
        return gbt, pipeline_gbt
    else:
        return gbt, pipeline_gbt

In [ ]:
%%spark

def fit_pipeline(pipeline, training_dataset):
    model_pipeline = pipeline.fit(training_dataset)
    return model_pipeline

def persist_modelpersist_(pipeline_model, model_name, model_save_path=model_save_path):
    pipeline_model.write().overwrite().save(model_save_path + model_name)

In [ ]:
%%spark

lr, rf, gbt, pipeline_lr, pipeline_rf, pipeline_gbt = create_pipeline(target, "lr", "rf", "gbt")

In [ ]:
%%spark

# Logistic Regression
start = time.time()
model_pipeline_lr = fit_pipeline(pipeline_lr, df_train)
end = time.time()

print('Logistic Regression Training Time:', end - start)

Logistic Regression Training Time: 14.408434391021729

In [ ]:
%%spark

# Random Forest
start = time.time()
model_pipeline_rf = fit_pipeline(pipeline_rf, df_train)
end = time.time()

print('Random Forest Training Time:', end - start)

Random Forest Training Time: 42.46438813209534

In [ ]:
%%spark

# Gradient Boosting
start = time.time()
model_pipeline_gbt = fit_pipeline(pipeline_gbt, df_train)
end = time.time()

print('Gradient Boosting Training Time:', end - start)

Gradient Boosting Training Time: 390.45907402038574

In [ ]:
%%spark

# # Cross Validation

# exclude_cols = ('servicecalendardate','ln_no',target)
# cat_cols = [i[0] for i in df_train.dtypes if ((i[1]=='string') & (~i[0].endswith(exclude_cols)))]
# num_cols = [i[0] for i in df_train.dtypes if ((i[1].startswith(('int', 'double'))) & (~i[0].endswith(exclude_cols)))]

# stages = []

# for col in cat_cols:

#     #Category indexing with StringIndexer
#     indexer = StringIndexer(inputCol = col, outputCol = col+'_idx').setHandleInvalid('keep')
#     stages += [indexer]

# #assemblerInputs = [c+'_vec' for c in cat_cols] + num_cols
# assemblerInputs = [c+'_idx' for c in cat_cols] + num_cols
# assembler = VectorAssembler(inputCols = assemblerInputs, outputCol = 'vectFeatures')

# stages += [assembler]

# gbt = GBTClassifier(maxBins=52, maxIter=100, featuresCol='vectFeatures', labelCol='label')

# pipeline_gbt = Pipeline(stages = stages + [gbt])

paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [10])\
    .build()

df_train = df_train.withColumnRenamed(target,'label')

crossval = CrossValidator(estimator=pipeline_gbt,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5)

In [ ]:
%%spark
cv.grid_scores_
cv.best_estimator_
cv.bestModel

### Save Models

In [ ]:
%%spark

persist_modelpersist_(model_pipeline_lr, 'MM_v22_LR')
persist_modelpersist_(model_pipeline_rf, 'MM_v22_RF')
persist_modelpersist_(model_pipeline_gbt, 'MM_v22_GBT')

### Read Models

In [ ]:
%%spark

model_pipeline_lr = PipelineModel.load(path = '/uat/DataScience/Maestro/Mover Refi/Model/MM_v22_LR')
model_pipeline_rf = PipelineModel.load(path = '/uat/DataScience/Maestro/Mover Refi/Model/MM_v22_RF')
model_pipeline_gbt = PipelineModel.load(path = '/uat/DataScience/Maestro/Mover Refi/Model/MM_v22_GBT')

# 3. Variable Importance

In [ ]:
%%spark
def extractFeatureImp(model_pipeline, df_train, featuresCol):

    featureImp = model_pipeline.stages[-1].featureImportances
    transformed = model_pipeline.transform(df_train)
    list_extract = []

    for i in transformed.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + transformed.schema[featuresCol].metadata["ml_attr"]["attrs"][i]

    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])

    return(varlist.sort_values('score', ascending = False))
    return varlist

In [ ]:
%%spark

# NEW
# GBT
gbt_feature_importance = extractFeatureImp(model_pipeline_gbt, df_train, "vectFeatures")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(gbt_feature_importance)

    idx                         name  \
7    16                 ageinmon_exp   
5    14           ratespread_min_exp   
6    15                ln_ann_int_rt   
8    17              currentcltv_exp   
18    4            og_mtg_am_exp_idx   
22    8                is_viable_idx   
16    2  loantypedescription_exp_idx   
9    18                orig_fico_exp   
13   22                    ln_tr_exp   
19    5      ln_purpose_type_exp_idx   
20    6  og_occupy_stat_type_exp_idx   
21    7                cancelflg_idx   
17    3     loanamortizationtype_idx   
14    0           home_value_exp_idx   
15    1    mosaic_group_refi_exp_idx   
0     9                   ct_age_exp   
11   20           LiveYears_long_exp   
10   19          LiveYears_short_exp   
4    13       personnum_per_room_exp   
12   21         issingleborrower_exp   
2    11                     ct_2_exp   
3    12                 p_edu_hs_exp   
1    10                     ct_1_exp   

                                       

In [ ]:
%%spark
# Old
# GBT
gbt_feature_importance = extractFeatureImp(model_pipeline_gbt, df_train, "vectFeatures")

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(gbt_feature_importance)

    idx                         name  \
5    13                 ageinmon_exp   
7    15              currentcltv_exp   
4    12           ratespread_min_exp   
6    14                og_mtg_am_exp   
8    16                orig_fico_exp   
20    6      ln_purpose_type_exp_idx   
18    4  loantypedescription_exp_idx   
13   21                    ln_tr_exp   
1     9               home_value_exp   
21    7  og_occupy_stat_type_exp_idx   
0     8                   ct_age_exp   
19    5     loanamortizationtype_idx   
10   18           LiveYears_long_exp   
17    3    mosaic_group_refi_exp_idx   
16    2                 ct_3_exp_idx   
3    11       personnum_per_room_exp   
9    17          LiveYears_short_exp   
12   20         issingleborrower_exp   
15    1                 ct_2_exp_idx   
2    10                 p_edu_hs_exp   
14    0                 ct_1_exp_idx   
11   19            LiveYears_grp_exp   

                                                 vals     score  
5            

# 4. ROC

### Predictions

In [ ]:
%%spark

def predict(model_pipeline, data):

    _scoreUdf = udf(lambda v: float(v[1]), DoubleType())

    prediction = model_pipeline.transform(data)
    pred_df = prediction.withColumn('pred', _scoreUdf(prediction['probability']))
    return pred_df

### ROC

In [ ]:
%%spark

def roc_auc(df, target):

    pred_df_lr = predict(model_pipeline_lr, df)
    pred_df_rf = predict(model_pipeline_rf, df)
    pred_df_gbt = predict(model_pipeline_gbt, df)

    ### Calculate ROC ###
    evalPred_lr = pred_df_lr.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')
    evalPred_rf = pred_df_rf.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')
    evalPred_gbt = pred_df_gbt.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')

    evaluatorLR = BinaryClassificationEvaluator()
    evaluatorRF = BinaryClassificationEvaluator()
    evaluatorGBT = BinaryClassificationEvaluator()

    print("Test Area Under ROC - LR: " + str(evaluatorLR.evaluate(evalPred_lr, {evaluatorLR.metricName: "areaUnderROC"})))
    print("Test Area Under ROC - RF: " + str(evaluatorRF.evaluate(evalPred_rf, {evaluatorRF.metricName: "areaUnderROC"})))
    print("Test Area Under ROC - GBT: " + str(evaluatorGBT.evaluate(evalPred_gbt, {evaluatorGBT.metricName: "areaUnderROC"})))
    print("\n")

In [ ]:
%%spark

# NEW

test_months = ['2020-09-30', '2020-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))

for df in [df_train, df_validate, df_test]:
    roc_auc(df, target)

Test Area Under ROC - LR: 0.6648597867127097
Test Area Under ROC - RF: 0.7108606500365694
Test Area Under ROC - GBT: 0.7530294754127758


Test Area Under ROC - LR: 0.6653141191131351
Test Area Under ROC - RF: 0.710487947462101
Test Area Under ROC - GBT: 0.7489090437143862


Test Area Under ROC - LR: 0.6525553510949061
Test Area Under ROC - RF: 0.709763298733102
Test Area Under ROC - GBT: 0.7351943021338879

In [ ]:
%%spark

# Compare with old model

monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'


df202006 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202006.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202009 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202009.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202012 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202012.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')


df_preprocessed = df202006.union(df202009).union(df202012)\
                            .drop('borrowerage_bucket','p_married_exp','p_31_50_exp')

target = 'refipayoff'

actual = spark.read.parquet(actual_path + 'Actual_payoff_20210405.parquet')\
                .select('servicecalendardate', 'ln_no', 'refipayoff')

df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()

test_months = ['2020-09-30', '2020-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))


old_model = PipelineModel.load(path = '/dev/projects/retention_models/refi_payoff_V1_4_legal/models/refiV1_4_GBT_20200513')

predictions = old_model.transform(df_test)


### Calculate ROC ###
evalPred_gbt = predictions.select(target, 'rawPrediction', 'prediction', 'probability')\
                          .withColumnRenamed(target, 'label')

evaluatorGBT = BinaryClassificationEvaluator()

print("Test Area Under ROC - GBT: " + str(evaluatorGBT.evaluate(evalPred_gbt, {evaluatorGBT.metricName: "areaUnderROC"})))
print("\n")



# test_months = ['2020-09-30', '2020-12-31']
# df_test = df_all.where(col('servicecalendardate').isin(test_months))

# for df in [df_train, df_validate, df_test]:
#     roc_auc(df, target)

Test Area Under ROC - GBT: 0.6925490429475004

In [ ]:
%%spark

test_months = ['2017-09-30', '2019-09-30', '2019-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))

for df in [df_train, df_validate, df_test]:
    roc_auc(df, target)

Test Area Under ROC - LR: 0.6332322703577062
Test Area Under ROC - RF: 0.680734951367731
Test Area Under ROC - GBT: 0.7252698287642437


Test Area Under ROC - LR: 0.6343274599427369
Test Area Under ROC - RF: 0.6792016999576734
Test Area Under ROC - GBT: 0.7196060082250103


Test Area Under ROC - LR: 0.6107240826340268
Test Area Under ROC - RF: 0.6768353729994416
Test Area Under ROC - GBT: 0.708076291895596

# 5. Capture Rate

In [ ]:
%%spark

df_train.groupBy('servicecalendardate').count().show()

+-------------------+------+
|servicecalendardate| count|
+-------------------+------+
|         2018-12-31| 85017|
|         2019-06-30|114434|
|         2019-09-30|124194|
|         2019-03-31| 94184|
|         2019-12-31|131170|
|         2020-06-30|170199|
|         2020-03-31|155802|
+-------------------+------+

In [ ]:
%%spark

# Decile for Sept. and Dec.

# NEW
test_months = ['2020-09-30']
df_test = df_all.where(col('servicecalendardate').isin(test_months))

predictions = model_pipeline_gbt.transform(df_test)
predictions = predictions.select('refipayoff','probability','prediction', F.ntile(10).over(Window.partitionBy().orderBy('probability')).alias("decile"))
dcount = predictions.groupby('decile').count()
pcount = predictions.groupby('decile').sum('refipayoff')

window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
total_target = pcount.select(F.sum('sum(refipayoff)')).collect()[0][0]
dcount.join(pcount, on=['decile'], how='left')\
      .withColumnRenamed('sum(refipayoff)','total_refipayoff')\
      .withColumn('cum_sum', F.sum('total_refipayoff').over(window_cumsum) / total_target).show()

+------+------+----------------+-------------------+
|decile| count|total_refipayoff|            cum_sum|
+------+------+----------------+-------------------+
|     1|206191|           41577|0.24497696178366465|
|     2|206191|           30173|0.42276010794376556|
|     3|206191|           25327| 0.5719900069527097|
|     4|206191|           21278| 0.6973626839816637|
|     5|206191|           17699| 0.8016474386924192|
|     6|206191|           13714| 0.8824520675473433|
|     7|206191|           10091| 0.9419095204987096|
|     8|206191|            6539| 0.9804381385592571|
|     9|206190|            2721| 0.9964706159629503|
|    10|206190|             599|                1.0|
+------+------+----------------+-------------------+

In [ ]:
%%spark

# NEW
test_months = ['2020-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))

predictions = model_pipeline_gbt.transform(df_test)
predictions = predictions.select('refipayoff','probability','prediction', F.ntile(10).over(Window.partitionBy().orderBy('probability')).alias("decile"))
dcount = predictions.groupby('decile').count()
pcount = predictions.groupby('decile').sum('refipayoff')

window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
total_target = pcount.select(F.sum('sum(refipayoff)')).collect()[0][0]
dcount.join(pcount, on=['decile'], how='left')\
      .withColumnRenamed('sum(refipayoff)','total_refipayoff')\
      .withColumn('cum_sum', F.sum('total_refipayoff').over(window_cumsum) / total_target).show()

+------+------+----------------+------------------+
|decile| count|total_refipayoff|           cum_sum|
+------+------+----------------+------------------+
|     1|217605|           31619| 0.272366267551038|
|     2|217605|           22110|0.4628219484882419|
|     3|217605|           17671|0.6150400551296408|
|     4|217605|           14044|0.7360151606512189|
|     5|217605|           10992|0.8307003187182358|
|     6|217604|            8252|0.9017830993194935|
|     7|217604|            5964|0.9531570333362047|
|     8|217604|            3428|0.9826858471875269|
|     9|217604|            1559|0.9961150831251615|
|    10|217604|             451|               1.0|
+------+------+----------------+------------------+

In [ ]:
%%spark

# Compare with old model Spetember

monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'


df202006 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202006.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202009 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202009.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202012 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202012.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')


df_preprocessed = df202006.union(df202009).union(df202012)\
                            .drop('borrowerage_bucket','p_married_exp','p_31_50_exp')

target = 'refipayoff'

actual = spark.read.parquet(actual_path + 'Actual_payoff_20210405.parquet')\
                .select('servicecalendardate', 'ln_no', 'refipayoff')

df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()

test_months = ['2020-09-30']
df_test = df_all.where(col('servicecalendardate').isin(test_months))


old_model = PipelineModel.load(path = '/dev/projects/retention_models/refi_payoff_V1_4_legal/models/refiV1_4_GBT_20200513')

predictions = old_model.transform(df_test)
predictions = predictions.select('refipayoff','probability','prediction', F.ntile(10).over(Window.partitionBy().orderBy('probability')).alias("decile"))
dcount = predictions.groupby('decile').count()
pcount = predictions.groupby('decile').sum('refipayoff')

print()
window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
total_target = pcount.select(F.sum('sum(refipayoff)')).collect()[0][0]
dcount.join(pcount, on=['decile'], how='left')\
      .withColumnRenamed('sum(refipayoff)','total_refipayoff')\
      .withColumn('cum_sum', F.sum('total_refipayoff').over(window_cumsum) / total_target).show()


+------+------+----------------+-------------------+
|decile| count|total_refipayoff|            cum_sum|
+------+------+----------------+-------------------+
|     1|206191|         35526.0|0.20666666666666667|
|     2|206191|         28205.0|0.37074461896451427|
|     3|206191|         22857.0| 0.5037114601512507|
|     4|206191|         20748.0| 0.6244095404304828|
|     5|206191|         19208.0| 0.7361489237929029|
|     6|206191|         17246.0| 0.8364746945898779|
|     7|206191|         13892.0| 0.9172891215823153|
|     8|206191|          9147.0|  0.970500290866783|
|     9|206190|          4137.0|   0.99456660849331|
|    10|206190|           934.0|                1.0|
+------+------+----------------+-------------------+

In [ ]:
%%spark

# Compare with old model December

monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'


df202006 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202006.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202009 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202009.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202012 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202012.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')


df_preprocessed = df202006.union(df202009).union(df202012)\
                            .drop('borrowerage_bucket','p_married_exp','p_31_50_exp')

target = 'refipayoff'

actual = spark.read.parquet(actual_path + 'Actual_payoff_20210405.parquet')\
                .select('servicecalendardate', 'ln_no', 'refipayoff')

df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()

test_months = ['2020-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))


old_model = PipelineModel.load(path = '/dev/projects/retention_models/refi_payoff_V1_4_legal/models/refiV1_4_GBT_20200513')

predictions = old_model.transform(df_test)
predictions = predictions.select('refipayoff','probability','prediction', F.ntile(10).over(Window.partitionBy().orderBy('probability')).alias("decile"))
dcount = predictions.groupby('decile').count()
pcount = predictions.groupby('decile').sum('refipayoff')

print()
window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
total_target = pcount.select(F.sum('sum(refipayoff)')).collect()[0][0]
dcount.join(pcount, on=['decile'], how='left')\
      .withColumnRenamed('sum(refipayoff)','total_refipayoff')\
      .withColumn('cum_sum', F.sum('total_refipayoff').over(window_cumsum) / total_target).show()


+------+------+----------------+-------------------+
|decile| count|total_refipayoff|            cum_sum|
+------+------+----------------+-------------------+
|     1|217605|         28124.0|  0.239292095635157|
|     2|217605|         20795.0|0.41622564451629374|
|     3|217605|         15646.0| 0.5493491023568451|
|     4|217605|         14679.0|  0.674244873649281|
|     5|217605|         13346.0| 0.7877988598655662|
|     6|217604|         11208.0| 0.8831617459372075|
|     7|217604|          7655.0| 0.9482940525823194|
|     8|217604|          4030.0| 0.9825831702544031|
|     9|217604|          1532.0| 0.9956181400493491|
|    10|217604|           515.0|                1.0|
+------+------+----------------+-------------------+

In [ ]:
%%spark

# Compare with old model

monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'


df202006 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202006.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202009 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202009.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')
df202012 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202012.parquet').drop('p_31_50_exp', 'p_married_exp', 'borrowerage_bucket')


df_preprocessed = df202006.union(df202009).union(df202012)\
                            .drop('borrowerage_bucket','p_married_exp','p_31_50_exp')

target = 'refipayoff'

actual = spark.read.parquet(actual_path + 'Actual_payoff_20210405.parquet')\
                .select('servicecalendardate', 'ln_no', 'refipayoff')

df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()

test_months = ['2020-09-30', '2020-12-31']
df_test = df_all.where(col('servicecalendardate').isin(test_months))


old_model = PipelineModel.load(path = '/dev/projects/retention_models/refi_payoff_V1_4_legal/models/refiV1_4_GBT_20200513')

predictions = old_model.transform(df_test)
predictions = predictions.select('refipayoff','probability','prediction', F.ntile(10).over(Window.partitionBy().orderBy('probability')).alias("decile"))
dcount = predictions.groupby('decile').count()
pcount = predictions.groupby('decile').sum('refipayoff')

print()
window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
total_target = pcount.select(F.sum('sum(refipayoff)')).collect()[0][0]
dcount.join(pcount, on=['decile'], how='left')\
      .withColumnRenamed('sum(refipayoff)','total_refipayoff')\
      .withColumn('cum_sum', F.sum('total_refipayoff').over(window_cumsum) / total_target).show()


+------+------+----------------+-------------------+
|decile| count|total_refipayoff|            cum_sum|
+------+------+----------------+-------------------+
|     1|423796|         39361.0|0.22355948087354102|
|     2|423796|         29803.0| 0.3928321926561213|
|     3|423796|         23271.0|  0.525004969755488|
|     4|423795|         21704.0| 0.6482776247408627|
|     5|423795|         19994.0| 0.7618379575724874|
|     6|423795|         17510.0| 0.8612898645386647|
|     7|423795|         12938.0| 0.9347740891148155|
|     8|423795|          7493.0| 0.9773322352540255|
|     9|423795|          3149.0| 0.9952176752903757|
|    10|423795|           842.0|                1.0|
+------+------+----------------+-------------------+

In [ ]:
%%spark

def roc_auc(df, target):

    pred_df_gbt = predict(old_model, df)

    ### Calculate ROC ###
    evalPred_gbt = pred_df_gbt.select(target, 'rawPrediction', 'prediction', 'probability')\
                                .withColumnRenamed(target, 'label')

    evaluatorGBT = BinaryClassificationEvaluator()

    print("Test Area Under ROC - GBT: " + str(evaluatorGBT.evaluate(evalPred_gbt, {evaluatorGBT.metricName: "areaUnderROC"})))

for df in [df_test]:
    roc_auc(df, target)

Test Area Under ROC - GBT: 0.6925490429475027

In [ ]:
%%spark
print(date_label)
deciles(df_all.filter(col('servicecalendardate') == date_label), model_pipeline_gbt, target).show()

2019-12-31
+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    180229|   25866.0|0.24701332187365707|
|     2|    180210|   17371.0| 0.4129016855273839|
|     3|    180207|   13264.0| 0.5395693071670725|
|     4|    180210|   11468.0| 0.6490856133314233|
|     5|    180246|    9632.0| 0.7410686148116316|
|     6|    180236|    8537.0|  0.822594661700807|
|     7|    180205|    7373.0| 0.8930048226137611|
|     8|    180240|    5954.0| 0.9498639163443633|
|     9|    180230|    3959.0| 0.9876712982858235|
|    10|    180230|    1291.0|                1.0|
+------+----------+----------+-------------------+

In [ ]:
%%spark

actual1 = spark.read.parquet(actual_path + 'Actual_payoff_20200505.parquet')\
                .select('servicecalendardate', 'ln_no', 'refipayoff')

df_all1 = df_preprocessed.join(actual1, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates().where(col('servicecalendardate') == '2019-12-31')

deciles(df_all1.filter(col('servicecalendardate') == date_label), model_pipeline_gbt, target).show()

+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    180219|   26262.0| 0.2452192425487413|
|     2|    180220|   17688.0|0.41037947262269364|
|     3|    180210|   13548.0|  0.536882796743109|
|     4|    180227|   11711.0| 0.6462332860237544|
|     5|    180237|    9875.0| 0.7384402778815269|
|     6|    180213|    8778.0| 0.8204041234033017|
|     7|    180229|    7604.0| 0.8914058414880107|
|     8|    180237|    6172.0| 0.9490363785762307|
|     9|    180214|    4091.0| 0.9872357511018152|
|    10|    180237|    1367.0|                1.0|
+------+----------+----------+-------------------+

# 6. Predictions

In [ ]:
%%spark

def fullMonthPrediction(df):

    pred_df_lr = predict(model_pipeline_lr, df)
    validationPredictionsLR = pred_df_lr.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'logRegProb')

    pred_df_rf = predict(model_pipeline_rf, df)
    validationPredictionsRF = pred_df_rf.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'randForProb')

    pred_df_gbt = predict(model_pipeline_gbt, df)
    validationPredictionsGBT = pred_df_gbt.select('ln_no', 'pred')\
                                            .withColumnRenamed('pred', 'gbtProb')

    combinedFinalPred = validationPredictionsLR.join(validationPredictionsRF, on='ln_no', how='left')\
                                                .join(validationPredictionsGBT, on='ln_no', how='left')\
                                                .dropDuplicates()

    return combinedFinalPred

In [ ]:
%%spark

def save_predictions_csv(df_pred, filename, result_path = result_path):

    df_pred.coalesce(1).write.csv(result_path + filename, header=True)

In [ ]:
%%spark

def pred_save(df, servicedate, filename, result_path = result_path):

    df1 = df.where(col('servicecalendardate') == servicedate)
    df_pred = fullMonthPrediction(df1)

    save_predictions_csv(df_pred, filename)

In [ ]:
%%spark

def pred_save_newmonth(preprocessed_file, filename, monthly_prep_path=monthly_prep_path, result_path=result_path):

    df = spark.read.parquet(monthly_prep_path + preprocessed_file)
    df_pred = fullMonthPrediction(df)

    print(df.count())
    save_predictions_csv(df_pred, filename)

In [ ]:
%%spark

# 2017-9
pred_save(df_test.where(col('servicecalendardate') == '2017-09-30'), '2017-09-30', 'pred_refiV1_4_sep17_20200514.csv')


# 2019-9
pred_save(df_test.where(col('servicecalendardate') == '2019-09-30'), '2019-09-30', 'pred_refiV1_4_sep19_20200514.csv')


# 2019-12
pred_save(df_test.where(col('servicecalendardate') == '2019-12-31'), '2019-12-31', 'pred_refiV1_4_dec19_20200514.csv')


In [ ]:
%%spark

print(df_all.where(col('servicecalendardate') == '2017-09-30').count())
print(spark.read.csv(result_path + 'pred_refiV1_4_sep17_20200514.csv', header = True).count())


print(df_all.where(col('servicecalendardate') == '2019-09-30').count())
print(spark.read.csv(result_path + 'pred_refiV1_4_sep19_20200514.csv', header = True).count())


print(df_all.where(col('servicecalendardate') == '2019-12-31').count())
print(spark.read.csv(result_path + 'pred_refiV1_4_dec19_20200514.csv', header = True).count())



1468441
1468441
1763391
1763391
1802243
1802243

In [ ]:
%%spark

df202003 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202003.parquet')
df_preprocessed = df202003.drop('borrowerage_bucket','p_married_exp','p_31_50_exp')
df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()
df_all.groupby('servicecalendardate').agg(count('ln_no'), sum(target)).orderBy(col('servicecalendardate').desc()).show()

+-------------------+------------+---------------+
|servicecalendardate|count(ln_no)|sum(refipayoff)|
+-------------------+------------+---------------+
|         2020-03-31|     1867297|        62522.0|
+-------------------+------------+---------------+

In [ ]:
%%spark

# 2020-03
pred_save(df_all.where(col('servicecalendardate') == '2020-03-31'), '2020-03-31', 'pred_refiV1_4_mar20_20200514.csv')

In [ ]:
%%spark

print(spark.read.csv(result_path + 'pred_refiV1_4_mar20_20200514.csv', header = True).count())

1867297

In [ ]:
%%spark

deciles(df_all.filter(col('servicecalendardate') == '2020-03-31'), model_pipeline_gbt, target).show()

+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    186704|   13777.0|0.22035443523879594|
|     2|    186751|   10976.0|0.39590864015866417|
|     3|    186740|    8277.0| 0.5282940404977448|
|     4|    186715|    6727.0| 0.6358881673650875|
|     5|    186717|    6492.0| 0.7397236172867151|
|     6|    186742|    5759.0| 0.8318351940117079|
|     7|    186735|    4741.0| 0.9076645020952625|
|     8|    186715|    3187.0|  0.958638559227152|
|     9|    186715|    2023.0| 0.9909951697002655|
|    10|    186763|     563.0|                1.0|
+------+----------+----------+-------------------+

In [ ]:
%%spark

df202004 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202004.parquet')
df_preprocessed = df202004.drop('borrowerage_bucket','p_married_exp','p_31_50_exp')
df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()

pred_save(df_all.where(col('servicecalendardate') == '2020-04-30'), '2020-04-30', 'pred_refiV1_4_apr20_20200514.csv')
print(spark.read.csv(result_path + 'pred_refiV1_4_apr20_20200514.csv', header = True).count())

deciles(df_all.filter(col('servicecalendardate') == '2020-04-30'), model_pipeline_gbt, target).show()

DataFrame[servicecalendardate: date, count(ln_no): bigint, sum(refipayoff): double]
1899099
+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    189904|    4044.0|0.23578800069966765|
|     2|    189889|    2937.0| 0.4070316599615183|
|     3|    189936|    2329.0| 0.5428254912250015|
|     4|    189899|    1907.0| 0.6540143431869861|
|     5|    189909|    1734.0| 0.7551163197481197|
|     6|    189901|    1503.0| 0.8427496938953997|
|     7|    189927|    1232.0| 0.9145822401026179|
|     8|    189906|     840.0|  0.963558976152994|
|     9|    189897|     519.0| 0.9938196023555478|
|    10|    189931|     106.0|                1.0|
+------+----------+----------+-------------------+

In [ ]:
%%spark

df_all.groupby('servicecalendardate').agg(count('ln_no'), sum(target)).orderBy(col('servicecalendardate').desc()).show()

+-------------------+------------+---------------+
|servicecalendardate|count(ln_no)|sum(refipayoff)|
+-------------------+------------+---------------+
|         2020-04-30|     1899099|        17151.0|
+-------------------+------------+---------------+

In [ ]:
%%spark

df202002 = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_202002.parquet')
df_preprocessed = df202002.drop('borrowerage_bucket','p_married_exp','p_31_50_exp')
df_all = df_preprocessed.join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()
pred_save(df_all.where(col('servicecalendardate') == '2020-02-29'), '2020-02-29', 'pred_refiV1_4_feb20_20200514.csv')

deciles(df_all.filter(col('servicecalendardate') == '2020-02-29'), model_pipeline_gbt, target).show()

+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    184673|   24445.0|0.23387644588168885|
|     2|    184677|   17463.0| 0.4009529185522527|
|     3|    184698|   13762.0| 0.5326202389950345|
|     4|    184681|   12389.0| 0.6511514432506387|
|     5|    184695|   10712.0| 0.7536380248945188|
|     6|    184689|    8754.0| 0.8373915289750385|
|     7|    184672|    7048.0|  0.904822954238861|
|     8|    184709|    5449.0| 0.9569560184077841|
|     9|    184661|    3506.0| 0.9904995168435051|
|    10|    184709|     993.0|                1.0|
+------+----------+----------+-------------------+

In [ ]:
%%spark

df_all = spark.read.csv(result_path + 'pred_refiV1_3_feb20_20200305.csv', header = True)
df_all.printSchema()

root
 |-- ln_no: string (nullable = true)
 |-- og_note_dt: string (nullable = true)
 |-- loantypedescription: string (nullable = true)
 |-- loanamortizationtype: string (nullable = true)
 |-- investornameshort: string (nullable = true)
 |-- loan_term: string (nullable = true)
 |-- loan_age: string (nullable = true)
 |-- currentcltv: string (nullable = true)
 |-- loan_interest_rate: string (nullable = true)
 |-- logRegProb: string (nullable = true)
 |-- randForProb: string (nullable = true)
 |-- gbtProb: string (nullable = true)

In [ ]:
%%spark

df = df_all.withColumn('pred', col('gbtProb').cast('double'))\
                .withColumn('servicecalendardate', lit('2020-02-29'))\
                .join(actual, on=['servicecalendardate', 'ln_no'], how='inner').dropDuplicates()
pred_df = QuantileDiscretizer(numBuckets=10, inputCol="pred", outputCol="decile", relativeError=0.00001,
                             handleInvalid="error").fit(df).transform(df)
pred_df = pred_df.withColumn('decile', (10 - F.col('decile')).cast('int'))

window_cumsum = Window.orderBy('decile').rangeBetween(Window.unboundedPreceding, 0)
total_target = pred_df.select(F.sum(target)).collect()[0][0]
df_out = pred_df\
        .groupBy('decile', )\
        .agg(F.count('ln_no').alias('decile_cnt'), F.sum(target).alias('payoff_cnt'))\
        .withColumn('cum_sum', F.sum('payoff_cnt').over(window_cumsum) / total_target)\
        .sort('decile')
df_out.show()

+------+----------+----------+-------------------+
|decile|decile_cnt|payoff_cnt|            cum_sum|
+------+----------+----------+-------------------+
|     1|    184678|   21641.0|0.20704930109738712|
|     2|    184685|   16132.0|0.36139149070521714|
|     3|    184684|   13726.0|0.49271438275561846|
|     4|    184685|   12516.0| 0.6124606538398982|
|     5|    184675|   10899.0| 0.7167363496330881|
|     6|    184713|    9263.0|  0.805359688483654|
|     7|    184651|    8106.0| 0.8829134815013251|
|     8|    184714|    6831.0| 0.9482687689555209|
|     9|    184665|    4287.0| 0.9892844500148296|
|    10|    184714|    1120.0|                1.0|
+------+----------+----------+-------------------+